# A Quick Introduction to Optuna

This Jupyter notebook goes through the basic usage of Optuna.

- Install Optuna
- Write a training algorithm that involves hyperparameters
  - Read train/valid data
  - Define and train model
  - Evaluate model
- Use Optuna to tune the hyperparameters (hyperparameter optimization, HPO)
- Visualize HPO

## Install `optuna`

Optuna can be installed via `pip` or `conda`.

In [1]:
!pip install --quiet optuna

     |████████████████████████████████| 296kB 6.8MB/s 
     |████████████████████████████████| 1.2MB 9.7MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 143kB 19.4MB/s 
     |████████████████████████████████| 112kB 20.7MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 


In [2]:
import optuna

optuna.__version__

'2.7.0'

## Optimize Hyperparameters

### Define a simple scikit-learn model

We start with a simple random forest model to classify flowers in the Iris dataset. We define a function called `objective` that encapsulates the whole training process and outputs the accuracy of the model.

In [3]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection

def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.
    
    clf = sklearn.ensemble.RandomForestClassifier(    
        n_estimators=5, max_depth=3)  # Define the model.
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()  # Train and evaluate the model.

print('Accuracy: {}'.format(objective()))

Accuracy: 0.9533333333333333


### Optimize hyperparameters of the model

The hyperparameters of the above algorithm are `n_estimators` and `max_depth` for which we can try different values to see if the model accuracy can be improved. The `objective` function is modified to accept a trial object. This trial has several methods for sampling hyperparameters. We create a study to run the hyperparameter optimization and finally read the best hyperparameters.

In [4]:
import optuna

def objective(trial):
    iris = sklearn.datasets.load_iris()
    
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))
    
    clf = sklearn.ensemble.RandomForestClassifier(
        n_estimators=n_estimators, max_depth=max_depth)
    
    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-05-16 04:40:40,539] A new study created in memory with name: no-name-17eca62e-7585-47f5-abaf-6c3054e71346
[I 2021-05-16 04:40:40,624] Trial 0 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 15, 'max_depth': 16.20369693502995}. Best is trial 0 with value: 0.9466666666666667.
[I 2021-05-16 04:40:40,717] Trial 1 finished with value: 0.96 and parameters: {'n_estimators': 18, 'max_depth': 6.422970928839076}. Best is trial 1 with value: 0.96.
[I 2021-05-16 04:40:40,785] Trial 2 finished with value: 0.9333333333333332 and parameters: {'n_estimators': 11, 'max_depth': 1.2628618818036534}. Best is trial 1 with value: 0.96.
[I 2021-05-16 04:40:40,867] Trial 3 finished with value: 0.96 and parameters: {'n_estimators': 15, 'max_depth': 1.9398806300221108}. Best is trial 1 with value: 0.96.
[I 2021-05-16 04:40:40,936] Trial 4 finished with value: 0.9466666666666667 and parameters: {'n_estimators': 12, 'max_depth': 3.7603335127486024}. Best is trial 1 with value: 0.9

Accuracy: 0.9733333333333333
Best hyperparameters: {'n_estimators': 9, 'max_depth': 2.6713771018195085}


It is possible to condition hyperparameters using Python `if` statements. We can for instance include another classifier, a support vector machine, in our HPO and define hyperparameters specific to the random forest model and the support vector machine.

In [5]:
import sklearn.svm

def objective(trial):
    iris = sklearn.datasets.load_iris()

    classifier = trial.suggest_categorical('classifier', ['RandomForest', 'SVC'])
    
    if classifier == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 2, 20)
        max_depth = int(trial.suggest_float('max_depth', 1, 32, log=True))

        clf = sklearn.ensemble.RandomForestClassifier(
            n_estimators=n_estimators, max_depth=max_depth)
    else:
        c = trial.suggest_float('svc_c', 1e-10, 1e10, log=True)
        
        clf = sklearn.svm.SVC(C=c, gamma='auto')

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

trial = study.best_trial

print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-05-16 04:40:54,285] A new study created in memory with name: no-name-353599cc-8144-4548-b406-c462f311d7e1
[I 2021-05-16 04:40:54,312] Trial 0 finished with value: 0.32 and parameters: {'classifier': 'SVC', 'svc_c': 0.002288428054700031}. Best is trial 0 with value: 0.32.
[I 2021-05-16 04:40:54,407] Trial 1 finished with value: 0.9533333333333333 and parameters: {'classifier': 'RandomForest', 'n_estimators': 16, 'max_depth': 2.015171842115937}. Best is trial 1 with value: 0.9533333333333333.
[I 2021-05-16 04:40:54,430] Trial 2 finished with value: 0.9666666666666667 and parameters: {'classifier': 'SVC', 'svc_c': 27.103869761038688}. Best is trial 2 with value: 0.9666666666666667.
[I 2021-05-16 04:40:54,461] Trial 3 finished with value: 0.9733333333333333 and parameters: {'classifier': 'RandomForest', 'n_estimators': 3, 'max_depth': 7.3267242822248075}. Best is trial 3 with value: 0.9733333333333333.
[I 2021-05-16 04:40:54,478] Trial 4 finished with value: 0.96 and parameters: {'

Accuracy: 0.9866666666666667
Best hyperparameters: {'classifier': 'SVC', 'svc_c': 3.899817601098268}


### Plotting the study

Plotting the optimization history of the study.

In [6]:
optuna.visualization.plot_optimization_history(study)

Plotting the accuracies for each hyperparameter for each trial.

In [7]:
optuna.visualization.plot_slice(study)

Plotting the accuracy surface for the hyperparameters involved in the random forest model.

In [8]:
optuna.visualization.plot_contour(study, params=['n_estimators', 'max_depth'])